In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_ta as ta

%matplotlib widget
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

product_data = {
    "SPY": {
        "directory_path": "data/spy-options-data-2020-2022",
        "df_raw": None,
        "df_final": None,
        "df_option_history": None,
    },
    "QQQ": {
        "directory_path": "data/qqq-options-data-2020-2022",
        "df_raw": None,
        "df_final": None,
        "df_option_history": None,
    },
}

for product_id in product_data.keys():
    dfs = []

    for dirname, _, filenames in os.walk(product_data[product_id]["directory_path"]):
        for filename in filenames:
            data_path = os.path.join(dirname, filename)
            print(data_path)
            dfs.append(pd.read_csv(data_path, low_memory=False))

    df_raw = pd.concat(dfs)
    columns = df_raw.columns
    columns = [s.replace("[", "") for s in columns]
    columns = [s.replace("]", "") for s in columns]
    columns = [s.replace(" ", "") for s in columns]
    df_raw.columns = columns

    product_data[product_id]["df_raw"] = df_raw

data/spy-options-data-2020-2022/spy_2020_2022.csv
data/qqq-options-data-2020-2022/qqq_2020_2022.csv


In [2]:
for product_id in product_data.keys():
    df_raw = product_data[product_id]["df_raw"]
    date_columns = ["QUOTE_READTIME", "QUOTE_DATE", "EXPIRE_DATE"]
    numeric_cols = df_raw.columns.to_list()
    numeric_cols.remove("QUOTE_READTIME")
    numeric_cols.remove("QUOTE_DATE")
    numeric_cols.remove("EXPIRE_DATE")

    df_numeric = df_raw.drop(columns=date_columns)

    for i in numeric_cols:
        df_numeric[i] = pd.to_numeric(df_numeric[i], errors="coerce")

    df_final = df_numeric.drop(columns=["C_SIZE", "P_SIZE"])
    product_data[product_id]["df_final"] = df_final

In [3]:
for product_id in product_data.keys():
    df_final = product_data[product_id]["df_final"]
    df_option_history = df_final.copy()

    df_option_history["EXPIRE_UNIX"] = pd.to_datetime(df_option_history.EXPIRE_UNIX, unit="s", utc=True)
    df_option_history["QUOTE_UNIXTIME"] = pd.to_datetime(df_option_history.QUOTE_UNIXTIME, unit="s", utc=True).apply(
        lambda x: pd.Timestamp(x).round(freq="D")
    )

    df_option_history.set_index(pd.DatetimeIndex(df_option_history.QUOTE_UNIXTIME), inplace=True)
    df_option_history.drop(columns=["QUOTE_UNIXTIME"], inplace=True)
    df_option_history.sort_index(inplace=True)

    df_option_history["UNDERLYING_PRODUCT_ID"] = product_id
    df_option_history["OPTION_ID"] = (
        df_option_history["UNDERLYING_PRODUCT_ID"]
        + df_option_history["EXPIRE_UNIX"].astype(str)
        + df_option_history["STRIKE"].astype(str)
    )
    df_option_history["OPTION_ID"] = df_option_history["OPTION_ID"].apply(lambda x: hash(x))

    def get_df_cnn_fear_greed_index():
        print("Retrieving historical ETF fear and greed index")
        import json

        with open("cnn_fear_greed_index_data.json") as f:
            cnn_fear_greed_index_data = json.load(f)

        df_fear_greed_index = pd.DataFrame(
            data=cnn_fear_greed_index_data["data"]["c:50108"]["series"][0], columns=["x", "y"]
        )
        df_fear_greed_index.set_index(
            pd.DatetimeIndex([pd.Timestamp(x, unit="s", tz="UTC") for x in df_fear_greed_index.x]),
            inplace=True,
        )
        df_fear_greed_index.rename(columns={"y": "fear_greed_index"}, inplace=True)
        df_fear_greed_index["fear_greed_index"] = df_fear_greed_index.fear_greed_index.astype(float).round()
        return df_fear_greed_index

    df_fear_greed_index = get_df_cnn_fear_greed_index()
    df_option_history = df_option_history.join(df_fear_greed_index, how="inner")

    df_history = df_option_history.groupby(df_option_history.index).first()[["UNDERLYING_LAST", "fear_greed_index"]]
    # df_history = df_option_history.groupby(df_option_history.index).first()[["UNDERLYING_LAST"]]
    df_history.rename(columns={"UNDERLYING_LAST": "close"}, inplace=True)

    CustomStrategy = ta.Strategy(
        name="RSI",
        ta=[
            {"kind": "rsi", "length": 14},
        ],
    )
    df_history.ta.strategy(CustomStrategy)

    for column in ["close", "fear_greed_index", "RSI_14"]:
        for days in [14]:
            periods = days
            df_history[f"{column}_min_{days}"] = df_history[column].rolling(window=periods, min_periods=periods).min()
            df_history[f"{column}_max_{days}"] = df_history[column].rolling(window=periods, min_periods=periods).max()

    # df_option_history = df_option_history.join(df_history, how="inner")
    df_option_history = df_option_history.join(df_history.drop(columns=["fear_greed_index"]), how="inner")
    df_option_history["index"] = df_option_history.index
    df_option_history.sort_values(by=["index", "EXPIRE_UNIX", "DTE", "STRIKE_DISTANCE_PCT"], inplace=True)

    product_data[product_id]["df_option_history"] = df_option_history

Retrieving historical ETF fear and greed index
Retrieving historical ETF fear and greed index


In [4]:
df_option_history = pd.concat([product_data[product_id]["df_option_history"] for product_id in product_data.keys()])
df_option_history.sort_values(
    by=["index", "UNDERLYING_PRODUCT_ID", "EXPIRE_UNIX", "DTE", "STRIKE_DISTANCE_PCT"], inplace=True
)
df_option_history

,QUOTE_TIME_HOURS,UNDERLYING_LAST,EXPIRE_UNIX,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,C_BID,C_ASK,STRIKE,P_BID,P_ASK,P_LAST,P_DELTA,P_GAMMA,P_VEGA,P_THETA,P_RHO,P_IV,P_VOLUME,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,UNDERLYING_PRODUCT_ID,OPTION_ID,x,fear_greed_index,close,RSI_14,close_min_14,close_max_14,fear_greed_index_min_14,fear_greed_index_max_14,RSI_14_min_14,RSI_14_max_14,index
2021-05-18 00:00:00+00:00,16.0,324.44,2021-05-17 20:00:00+00:00,0.0,1.00000,0.00000,0.00000,-0.00035,0.00694,NaN,NaN,0.35,0.34,0.44,324.0,0.03,0.07,0.04,-0.18585,0.54261,0.03924,-0.05014,-0.00094,0.03353,NaN,0.4,0.001,QQQ,-8578305565234411172,2021-05-18,35.0,324.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-18 00:00:00+00:00
2021-05-18 00:00:00+00:00,16.0,324.44,2021-05-17 20:00:00+00:00,0.0,0.09826,0.40027,0.02551,-0.02006,0.00069,0.03015,NaN,0.02,0.01,0.04,325.0,0.61,0.76,0.79,-0.74789,0.38048,0.04665,-0.12534,-0.00433,0.05740,NaN,0.6,0.002,QQQ,-3912251940911432638,2021-05-18,35.0,324.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-18 00:00:00+00:00
2021-05-18 00:00:00+00:00,16.0,324.44,2021-05-17 20:00:00+00:00,0.0,0.03029,0.08312,0.01023,-0.01026,-0.00003,0.05774,NaN,0.03,0.02,0.02,326.0,1.56,1.71,1.84,-0.87522,0.14638,0.02973,-0.08046,-0.00473,0.09216,481.0,1.6,0.005,QQQ,-796774111719326768,2021-05-18,35.0,324.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-18 00:00:00+00:00
2021-05-18 00:00:00+00:00,16.0,324.44,2021-05-17 20:00:00+00:00,0.0,1.00000,0.00000,0.00000,-0.00017,0.00682,NaN,NaN,2.38,2.26,2.42,322.0,0.02,0.03,0.01,-0.02943,0.05143,0.00999,-0.01417,-0.00016,0.08914,NaN,2.4,0.008,QQQ,-8305044159569129464,2021-05-18,35.0,324.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-18 00:00:00+00:00
2021-05-18 00:00:00+00:00,16.0,324.44,2021-05-17 20:00:00+00:00,0.0,0.00953,0.01719,0.00381,-0.00535,-0.00039,0.10426,NaN,0.01,0.00,0.01,328.0,3.54,3.70,4.03,-0.92770,0.04573,0.02007,-0.06009,-0.00385,0.16061,130.0,3.6,0.011,QQQ,-1070438577502752343,2021-05-18,35.0,324.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-05-18 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 00:00:00+00:00,16.0,382.44,2025-12-19 21:00:00+00:00,1085.0,0.10595,0.00150,1.19348,-0.01204,1.05129,0.17770,2.0,5.71,0.59,10.00,630.0,243.00,250.50,0.00,-1.00000,0.00000,0.00000,0.00000,0.00000,NaN,NaN,247.6,0.647,SPY,-2602091060095401525,2022-12-31,38.0,382.44,44.376002,376.71,401.95,36.0,61.0,37.972031,56.545485,2022-12-31 00:00:00+00:00
2022-12-31 00:00:00+00:00,16.0,382.44,2025-12-19 21:00:00+00:00,1085.0,0.09392,0.00140,1.09723,-0.01082,0.94207,0.17304,NaN,0.00,2.00,7.00,635.0,247.66,257.50,0.00,-0.89431,0.00307,0.72687,-0.00434,-0.20330,0.29557,NaN,252.6,0.660,SPY,-2872950415384401199,2022-12-31,38.0,382.44,44.376002,376.71,401.95,36.0,61.0,37.972031,56.545485,2022-12-31 00:00:00+00:00
2022-12-31 00:00:00+00:00,16.0,382.44,2025-12-19 21:00:00+00:00,1085.0,0.09279,0.00137,1.08956,-0.01114,0.93199,0.17558,NaN,0.00,2.00,7.00,640.0,253.00,262.50,0.00,-0.88536,0.00298,0.78662,-0.00524,-1.78777,0.30289,NaN,257.6,0.673,SPY,-6162682096078852249,2022-12-31,38.0,382.44,44.376002,376.71,401.95,36.0,61.0,37.972031,56.545485,2022-12-31 00:00:00+00:00
2022-12-31 00:00:00+00:00,16.0,382.44,2025-12-19 21:00:00+00:00,1085.0,0.09938,0.00147,1.14502,-0.01128,0.99068,0.18239,3.0,4.33,0.12,10.00,645.0,258.00,267.50,0.00,-0.88264,0.00287,0.79928,-0.00538,-1.82985,0.30699,NaN,262.6,0.687,SPY,3937355187600415841,2022-12-31,38.0,382.44,44.376002,376.71,401.95,36.0,61.0,37.972031,56.545485,2022-12-31 00:00:00+00:00


In [5]:
# for c in ["UNDERLYING_LAST", "fear_greed_index"]:
#     df = df_option_history[[c]]
#     df["date"] = df.index
#     df.drop_duplicates(subset=["date", c]).plot(x="date", y=c)

df_option_history.describe().astype(str)

,QUOTE_TIME_HOURS,UNDERLYING_LAST,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,C_BID,C_ASK,STRIKE,P_BID,P_ASK,P_LAST,P_DELTA,P_GAMMA,P_VEGA,P_THETA,P_RHO,P_IV,P_VOLUME,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,OPTION_ID,fear_greed_index,close,RSI_14,close_min_14,close_max_14,fear_greed_index_min_14,fear_greed_index_max_14,RSI_14_min_14,RSI_14_max_14
count,2924028.0,2924028.0,2924028.0,2923915.0,2923915.0,2923915.0,2923915.0,2923915.0,2760678.0,2288147.0,2921862.0,2921862.0,2921862.0,2924028.0,2921864.0,2921864.0,2921864.0,2923915.0,2923915.0,2923915.0,2923915.0,2923915.0,2727843.0,2268486.0,2924028.0,2924028.0,2924028.0,2924028.0,2924028.0,2793236.0,2802474.0,2802474.0,2802474.0,2802474.0,2678509.0,2678509.0
mean,16.0,384.4283184189752,138.02139746951772,0.5147727075171475,-0.1426652472695,0.37393658445953426,-0.062448612658028674,0.5338645020665787,0.32184403888465085,223.67174268086796,27.457221319829635,42.82091449219715,43.63370481905037,378.4349665940271,36.412099159988266,37.20279069799281,17.72762483811704,-0.4751130797851511,-1.3617365511890733,-3.372096385903825,-0.09526726047781825,-0.582160194400316,0.33483247641084907,278.73488617518467,70.2024498397416,0.1851790229094933,-5.510601608850764e+16,39.008525910148606,384.4283184189752,51.6879211539061,372.19830265686676,397.4107338301801,25.5486259640589,52.2128383706682,40.72111638826419,60.877541732984994
std,0.0,56.07916429168577,192.17584355995493,0.3861001036753277,253.33533046255502,11.651493477130716,0.07740579303640395,7.729401177572637,0.36249734941884854,2948.9633538167545,50.11363709421322,59.9099974008247,60.562646345379925,103.57771221293312,55.349329423001045,56.04450959839431,35.34904177146665,0.38379745483747785,1480.38156758852,51.643064341421855,0.698562963823025,1.3426167492294432,0.280395462162283,2867.3746998730007,69.38664370647663,0.18296795839302013,5.322665503534296e+18,16.58751018152316,56.07916429168577,12.344754413310964,55.83092688175122,54.13280441889287,13.969148762327524,14.340958582930437,10.346188286759945,9.91193022420065
min,16.0,260.05,0.0,0.0,-433189.70517,-2815.39969,-7.16086,-2878.27141,-0.0005,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,-1.0,-2211829.9611,-2199.50048,-43.52167,-36.5688,-0.0005,0.0,0.0,0.0,-9.223268179247174e+18,4.0,260.05,23.20999431509113,260.05,282.17,4.0,22.0,23.20999431509113,42.18860128883254
25%,16.0,349.86,16.0,0.06801,0.0005,0.0311,-0.08407,0.01039,0.19401,0.0,0.01,0.77,0.92,314.0,1.7,1.79,0.02,-0.89625,0.00057,0.05015,-0.08544,-0.56005,0.20922,0.0,21.6,0.057,-4.677348367868452e+18,27.0,349.86,41.85144511620841,339.57,368.47,17.0,41.0,32.63792421591388,52.672438402454766
50%,16.0,390.15,43.0,0.58257,0.0028,0.21219,-0.04248,0.1143,0.25148,1.0,3.33,17.41,18.06,379.0,13.69,14.28,2.98,-0.42047,0.00288,0.24047,-0.04357,-0.11167,0.27624,2.0,47.3,0.125,-6.743361482311614e+16,37.0,390.15,50.69687314399896,377.94,402.42,21.0,52.0,38.470403626970906,60.6994653162677
75%,16.0,432.88,191.04,0.9021,0.00684,0.57939,-0.01035,0.54251,0.3307,13.0,31.32,60.15,61.45,440.0,45.21,46.49,19.42,-0.07504,0.00676,0.62677,-0.01672,-0.00796,0.37669,30.0,98.6,0.26,4.520398356603582e+18,52.0,432.88,61.698535691456804,420.84,444.89,33.0,64.0,47.07514402308029,68.28616665486885
max,16.0,477.77,1096.0,1.0,2.26443,147.32807,0.0,263.72682,41.39908,282571.0,444.2,445.08,446.28,4898.0,4451.19,4455.5,359.21,0.0,280.89887,81.84637,0.0,0.0,12.17003,265102.0,4469.3,10.425,9.222556436732245e+18,77.0,477.77,83.36208171663986,464.72,477.77,63.0,77.0,72.8276648027665,83.36208171663986


In [14]:
min_dte_buy = 14
min_dte_sell = int(min_dte_buy / 2)
min_volume = 100
max_strike_distance_pct = 0.02
contract_fee = 0.66
total_contracts = 2

money = 2000
start_money = money
last_index = df_option_history.index.values[-1]

owned_options = []
trade_data = []

for index, row in df_option_history.iterrows():
    expire_date = row.EXPIRE_UNIX.strftime(format="%Y-%m-%d")
    index_date = index.strftime(format="%Y-%m-%d")
    is_last_index = index.to_numpy() == last_index

    good_call_buy = row.fear_greed_index >= row.fear_greed_index_max_14 - 1
    good_put_buy = row.fear_greed_index <= row.fear_greed_index_min_14 + 1

    call_bid_price = row.C_BID * 100
    put_bid_price = row.P_BID * 100

    owned_options_not_sold = []

    for option in owned_options:
        if option["OPTION_ID"] == row.OPTION_ID:
            if (
                option["side"] == "call"
                and not np.isnan(row.C_BID)
                and (
                    not good_call_buy
                    or row.DTE < min_dte_sell
                    or (option["DTE"] - row.DTE) > min_dte_sell
                    or is_last_index
                )
            ):
                money += (call_bid_price - contract_fee) * total_contracts
                print(
                    f"{index_date} {row.UNDERLYING_PRODUCT_ID} ${money:.0f} {len(owned_options)-1} sell call {expire_date} {row.UNDERLYING_LAST} {row.STRIKE} ${call_bid_price:.0f} {row.OPTION_ID}"
                )
                trade_data.append(
                    {
                        "date": index_date,
                        "money": money,
                        "side": "sell",
                        "price": call_bid_price,
                        "options": len(owned_options) - 1,
                    }
                )
            elif (
                option["side"] == "put"
                and not np.isnan(row.P_BID)
                and (
                    not good_put_buy
                    or row.DTE < min_dte_sell
                    or (option["DTE"] - row.DTE) > min_dte_sell
                    or is_last_index
                )
            ):
                money += (put_bid_price - contract_fee) * total_contracts
                print(
                    f"{index_date} {row.UNDERLYING_PRODUCT_ID} ${money:.0f} {len(owned_options)-1} sell put {expire_date} {row.UNDERLYING_LAST} {row.STRIKE} ${put_bid_price:.0f} {row.OPTION_ID}"
                )
                trade_data.append(
                    {
                        "date": index_date,
                        "money": money,
                        "side": "sell",
                        "price": put_bid_price,
                        "options": len(owned_options) - 1,
                    }
                )
            else:
                owned_options_not_sold.append(option)
        else:
            owned_options_not_sold.append(option)

    owned_options = owned_options_not_sold

    max_buy_price = money / total_contracts
    max_buy_amount = 10
    call_ask_price = row.C_ASK * 100
    put_ask_price = row.P_ASK * 100

    if not is_last_index and row.DTE > min_dte_buy and row.STRIKE_DISTANCE_PCT < max_strike_distance_pct:
        if (
            good_call_buy
            and not np.isnan(row.C_ASK)
            and row.C_VOLUME > min_volume
            and call_ask_price <= max_buy_price
            and money >= call_ask_price
            and len([x for x in owned_options if x["OPTION_ID"] == row.OPTION_ID]) == 0
            # and len([x for x in owned_options if x["side"] == "call" and x["purchase_date"] == index_date]) <= max_buy_amount
            # and row.UNDERLYING_LAST > row.STRIKE
        ):
            money -= (call_ask_price + contract_fee) * total_contracts
            option_to_buy = row.to_dict()
            option_to_buy["side"] = "call"
            option_to_buy["purchase_date"] = index_date
            owned_options.append(option_to_buy)
            print(
                f"{index_date} {row.UNDERLYING_PRODUCT_ID} ${money:.0f} {len(owned_options)} buy call {expire_date} {row.UNDERLYING_LAST} {row.STRIKE} ${call_ask_price:.0f} {row.OPTION_ID}"
            )
            trade_data.append(
                {
                    "date": index_date,
                    "money": money,
                    "side": "buy",
                    "price": call_ask_price,
                    "options": len(owned_options),
                }
            )
        elif (
            good_put_buy
            and not np.isnan(row.P_ASK)
            and row.P_VOLUME > min_volume
            and put_ask_price <= max_buy_price
            and money >= put_ask_price
            and len([x for x in owned_options if x["OPTION_ID"] == row.OPTION_ID]) == 0
            # and len([x for x in owned_options if x["side"] == "put" and x["purchase_date"] == index_date]) <= max_buy_amount
            # and row.UNDERLYING_LAST < row.STRIKE
        ):
            money -= (put_ask_price + contract_fee) * total_contracts
            option_to_buy = row.to_dict()
            option_to_buy["side"] = "put"
            option_to_buy["purchase_date"] = index_date
            owned_options.append(option_to_buy)
            print(
                f"{index_date} {row.UNDERLYING_PRODUCT_ID} ${money:.0f} {len(owned_options)} buy put {expire_date} {row.UNDERLYING_LAST} {row.STRIKE} ${put_ask_price:.0f} {row.OPTION_ID}"
            )
            trade_data.append(
                {
                    "date": index_date,
                    "money": money,
                    "side": "buy",
                    "price": put_ask_price,
                    "options": len(owned_options),
                }
            )

print(f"Start money: ${start_money:.2f}")
print(f"End money: ${money:.2f} ({(money-start_money)/start_money*100:.2f}%)")

2021-06-10 QQQ $1027 1 buy call 2021-06-25 336.86 337.0 $486 3479158678833100675
2021-06-10 QQQ $117 2 buy call 2021-06-25 336.86 337.5 $454 -3598241872408703621
2021-06-11 SPY $0 3 buy call 2021-06-25 423.61 431.0 $58 -3094602456089219840
2021-06-16 QQQ $1305 2 sell call 2021-06-25 342.15 337.5 $653 -3598241872408703621
2021-06-16 QQQ $2685 1 sell call 2021-06-25 342.15 337.0 $691 3479158678833100675
2021-06-16 SPY $2782 0 sell call 2021-06-25 424.51 431.0 $49 -3094602456089219840
2021-07-07 QQQ $2275 1 buy put 2021-07-21 360.13 354.0 $253 1455647653561036424
2021-07-07 QQQ $1305 2 buy put 2021-07-23 360.13 360.0 $484 -2619219960940882630
2021-07-07 QQQ $414 3 buy put 2021-07-23 360.13 359.0 $445 -3008072389610402244
2021-07-07 SPY $-1 4 buy put 2021-07-21 432.88 427.0 $207 4274915379359932434
2021-07-09 QQQ $679 3 sell put 2021-07-21 358.79 354.0 $341 1455647653561036424
2021-07-09 QQQ $1820 2 sell put 2021-07-23 358.79 359.0 $571 -3008072389610402244
2021-07-09 QQQ $3035 1 sell put 

KeyboardInterrupt: 

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(13, 4), dpi=80)

df_trades = pd.DataFrame(trade_data)

field_to_plot = "money"
# df_trades_plot = df_trades_plot.copy()
# df_trades_plot = df_trades.loc[(df_trades["side"] == "sell")]
df_trades_plot = df_trades.loc[(df_trades["side"] == "sell") & (df_trades["options"] == 0)]

# field_to_plot = "options"
# df_trades_plot = df_trades.groupby(["date"]).max("options").reset_index()

plt.clf()
plt.plot(pd.to_datetime(df_trades_plot["date"]), df_trades_plot[field_to_plot])
plt.xlabel("date")
plt.ylabel(field_to_plot)
plt.show()
df_trades.describe().astype(str)

In [ ]:
# SPY, max strike distance .05, fear and greed max+1/min-1, max purchase money / 10, max 10 options daily
# Options: 3304
# End money: $379182.36 (18859.12%)

# SPY, max strike distance .02, fear and greed max+1/min-1, max purchase money / 10, max 10 options daily
# Options: 2966
# End money: $369782.44 (18389.12%)

# SPY, max strike distance .02, fear and greed max+1/min-1, max purchase money, max 10 options daily
# Options: 3206
# End money: $427413.04 (21270.65%)

# SPY, max strike distance .01, fear and greed max+1/min-1, max purchase money, max 50 options daily
# Options: 8828
# End money: $1249933.52 (62396.68%)